# Final Project: Exploratory Analysis of the Disney Character Success Datasets

Author: Noah Sheridan

## Introduction

### Questions of Interest

This analysis will seek to answer two main questions articulated below.

* How have the contributions of Disney's business segments evolved over time?
    * I am interested to see whether or not Disney's main revenue driver is the Studio business segment, or whether other segments drive the lion's share of revenue. 
    * I am also interested to see if business segments that favour digital media (e.g. Media Networks and Interactive) have grown at a faster rate than the more traditional business segments (Studio, Parks & Resorts, and Consumer Products).

* Who are the top 5 disney directors based on inflation adjusted gross revenue and during which time periods were their movies released?
    * I am interested to see if the directors are clustered in similar time periods, or if they are spread out over the 80 years of data.

### Dataset description

The data source found [here](https://data.world/kgarrett/disney-character-success-00-16) describes the data as "Disney characters, box office success & annual gross income".

The dataset is composed of 5 main tables including `disney-characters.csv`, `disney-director.csv`, `disney-voice-actors.csv`, `disney_revenue_1991-2016.csv`, and `disney_movies_total_gross.csv`. This analysis will use the `disney_revenue_1991-2016.csv`, `disney-director.csv`, and `disney_movies_total_gross.csv` datasets, which are described in more detail below.

* `disney_revenue_1991-2016.csv`
    * This file provides the annual revenue from each of Disney's 5 business units (Studio Entertainment, Consumer Products, Interactive, Parks & Resorts, and Media Networks) along with the total annual revenue for the company from 1991 to 2016.
    
* `disney_movies_total_gross.csv`
    * This file provides the total gross revenue from each movie released, including data from 1937 to 2016. For each movie it includes the title, release date, genre, rating, total gross revenue in nominal dollars, and total gross revenue in real dollars (inflation adjusted to today).

* `disney-director.csv`
    * This file provides the directors associated with each Disney movie title. This analysis will use this data to enhance the `disney_movies_total_gross.csv` data.

## Exploratory Data Analysis

## Methods & Results

Let's begin by importing our key libraries and importing the datasets required

In [2]:
# Import librairies
import pandas as pd
import altair as alt

# Import datasets
revenue = pd.read_csv("data/disney_revenue_1991-2016.csv")
director = pd.read_csv("data/disney-director.csv")
gross = pd.read_csv("data/disney_movies_total_gross.csv")


Let's see what our revenue dataset looks like

In [3]:
# Display top 5 rows in our revenue dataframe
revenue.head()

,Year,Studio Entertainment[NI 1],Disney Consumer Products[NI 2],Disney Interactive[NI 3][Rev 1],Walt Disney Parks and Resorts,Disney Media Networks,Total
0,1991,2593.0,724.0,NaN,2794.0,NaN,6111
1,1992,3115.0,1081.0,NaN,3306.0,NaN,7502
2,1993,3673.4,1415.1,NaN,3440.7,NaN,8529
3,1994,4793.0,1798.2,NaN,3463.6,359,10414
4,1995,6001.5,2150.0,NaN,3959.8,414,12525


It appears that we have a number of NA entries - given we don't have any additional information, we likely want to fill them with zero.

In [4]:
# Fill NAs in all columns with zero
revenue = revenue.fillna(value=0)

Our dataframe also has some unconventional names for the columns, let's rename them to something easier to work with.

In [5]:
# Rename business segment columns to improve readability
revenue = revenue.rename(columns={revenue.columns[1]: 'Studio_entertainment',
                                  revenue.columns[2]: 'Consumer_products',
                                  revenue.columns[3]: 'Interactive',
                                  revenue.columns[4]: 'Parks_and_resorts',
                                  revenue.columns[5]: 'Media_networks'})

The media networks column has numbers stored as strings, so let's clean up that column and convert it to float.

In [6]:
# Remove commas and blank space from Media_networks column then convert to float to facilitate sum calculations, then refill any NAs with zero
revenue = revenue.assign(Media_networks=revenue['Media_networks'].str.strip())
revenue = revenue.assign(Media_networks=revenue['Media_networks'].str.replace(",", "").astype(float))
revenue = revenue.fillna(value=0)

To answer our first question, we will sum the total revenues from each business segment and compare the totals.

In [7]:
# Aggregate dataframe by summing each column
revenue.agg('sum')

Year                     52091.0
Studio_entertainment    161125.9
Consumer_products        62185.3
Interactive               8564.0
Parks_and_resorts       221328.1
Media_networks          296187.0
Total                   765952.0
dtype: float64

It appears that our initial hypothesis was incorrect. The **Studio Entertainment** segment generated 161B in revenue over the full timespan; however, both the **Media Networks** and **Parks & Resorts** segments generated over 200B each over the same timeframe. 

In order to understand the growth of each business segment over time, we will convert the annual revenues into a percent growth versus a baseline. We will select 1997 as the baseline year because it is the first year where every business segment generated revenue. 

In [8]:
# Filter dataframe on years 1997 and after
revenue_pct = revenue[revenue['Year'] >= 1997].reset_index(drop=True)

# Override columns with a calculation to determine percent change vs 2017 (revenue in given year / revenue in 1997 baseline year) - 100%
for columns in revenue_pct.iloc[:, 1:]:
    revenue_pct.loc[:, columns] = (revenue_pct[columns]/revenue_pct.loc[0, columns])-1
    
# Display dataframe to evaluate results
revenue_pct

,Year,Studio_entertainment,Consumer_products,Interactive,Parks_and_resorts,Media_networks,Total
0,1997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1998,-0.018908,-0.155738,0.494253,0.103311,0.095063,0.022382
2,1999,-0.062025,-0.198837,0.183908,0.217790,0.151794,0.041338
3,2000,-0.141384,-0.312004,1.114943,0.356801,0.474241,0.130334
4,2001,0.003295,-0.315177,-1.000000,0.198444,0.467188,0.147599
5,2002,-0.073915,-0.354839,-1.000000,0.334464,0.492334,0.128465
6,2003,0.054863,-0.380222,-1.000000,0.278819,0.677553,0.204156
7,2004,0.248102,-0.336066,-1.000000,0.545672,0.805888,0.368398
8,2005,0.086807,-0.437599,-1.000000,0.799561,1.024992,0.421439
9,2006,0.078499,-0.420148,-1.000000,0.979458,1.203005,0.525609


Before we visualize the data, let's drop the first row (because growth in year one will be zero) and drop the last row, as 2015 is the last year that includes data for all business segments.

In [9]:
# Drop baseline year (2017) and final year (2016) with missing data
revenue_pct = revenue_pct[(revenue_pct['Year'] >= 1998) & (revenue_pct['Year'] <= 2015)].reset_index(drop=True)

Now, let's plot the data using a bar chart and group each business segment together to view growth trends! In order to facilitate grouping business segments by colour, we first need to melt the data into a long dataframe where business segment is one unique column.

In [10]:
# Use melt to convert the wide dataframe into a long dataframe, with the old business segment columns converted into one column named Business_segment
revenue_melt = revenue_pct.melt(id_vars=['Year'], value_vars=revenue_pct.columns.tolist()[1:], var_name='Business_segment', value_name='Percent_growth')

In [11]:
# Use altair to plot a bar chart with Business segement on the colour/column grouping
Percent_growth_chart = (alt.Chart(revenue_melt)
                        .mark_bar()
                        .encode(x=alt.X('Year:O', title="Year"),
                                y=alt.Y('Percent_growth:Q', title="Percent Growth", axis=alt.Axis(format='%')),
                                color='Business_segment:N',
                                column='Business_segment:N')
                        .properties(title="Chart 1: Annual Percent Revenue Growth vs 1997 Baseline for Disney's Business Segments"))
Percent_growth_chart

alt.Chart(...)

It appears that our original hypothesis is essentially correct. Other than a few years in 2001-2007 with missing data, the **Interactive** (gaming) business segment showed the most growth from 1997-2015, ending 2015 at ~575% growth vs. 1997. The next fastest growing segment was **Media Networks**, showing steady annual growth and ending at ~250% growth vs. 1997.

***

Moving on to our second question, we need to join together the gross and director dataframes in order to attribute directors to the movies they directed. We will use an inner join, as there are some entries that do not have directors associated with the movie titles. We only want to analyze the movies that we have directors for. 

In [11]:
# Rename columns to standardize naming convention
gross = gross.rename(columns={gross.columns[0]: 'Movie_title',
                              gross.columns[1]: 'Release_date',
                              gross.columns[2]: 'Genre',
                              gross.columns[3]: 'MPAA_rating',
                              gross.columns[4]: 'Total_gross',
                              gross.columns[5]: 'Inflation_adjusted_gross'})

# Join gross revenue dataframe with director dataframe using inner join
gross_director = gross.merge(director, left_on="Movie_title", right_on="name", how='inner')
gross_director.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Movie_title               49 non-null     object
 1   Release_date              49 non-null     object
 2   Genre                     48 non-null     object
 3   MPAA_rating               42 non-null     object
 4   Total_gross               49 non-null     object
 5   Inflation_adjusted_gross  49 non-null     object
 6   name                      49 non-null     object
 7   director                  49 non-null     object
dtypes: object(8)
memory usage: 3.4+ KB


It appears all the data in our dataframe is of Dtype object / string. We want to convert the `Release_date` column to datetime and the `Inflation_adjusted_gross` column to float, so that we can perform operations more easily

In [12]:
# Convert Release_date column to datetime to facilitate min/max operations
gross_director = gross_director.assign(Release_date=pd.to_datetime(gross_director['Release_date']))

# Remove dollar signs and commas from Inflation_adjusted_gross column then convert to float to facilitate sum calculations, and to millions of dollars to improve readability
gross_director.loc[:, 'Inflation_adjusted_gross'] = gross_director['Inflation_adjusted_gross'].str.replace(",", "")
gross_director.loc[:, 'Inflation_adjusted_gross'] = gross_director['Inflation_adjusted_gross'].str.replace("$", "", regex=True).astype(float)/1000000

Now that we've cleaned up our data, we can analyze our data to answer our question. First we group by director, then aggregate in order to find the sum of `Inflation_adjusted_gross`, as well as the min and max of `Release_date`. We will also sort the data by sum of gross revenue and filter on the first 5 rows, which will isolate and display the top 5 directors by inflation adjusted gross revenue.

In [13]:
# Import our groupby and aggregating function
# Group by director and aggregate to understand the gross revenue per director, as well as the oldest and newest releases of their films. Sort by highest sum of gross revenue.
from group_agg_script import group_agg

gross_director_agg = group_agg(gross_director, "director", "Inflation_adjusted_gross", "Release_date", ['sum'], ['min', 'max'])

gross_director_agg.sort_values(by=('Inflation_adjusted_gross', 'sum'), ascending=False).iloc[0:5, :]

Inflation_adjusted_gross Release_date           
                                         sum          min        max
director                                                            
David Hand                       5228.953251   1937-12-21 1937-12-21
Wolfgang Reitherman              3432.919920   1961-01-25 2016-04-15
Ben Sharpsteen                   2188.229052   1940-02-09 1940-02-09
full credits                     2187.090808   1940-11-13 1940-11-13
Ron Clements                     1318.949600   1986-07-02 2016-11-23

It appears that, on a time value of money adjusted basis, some of the older directors have the highest revenue for disney. Let's visualize the data.

In [14]:
# Reset index to facilitate plotting
gross_director_plot_data = gross_director_agg.sort_values(by=('Inflation_adjusted_gross', 'sum'), ascending=False).iloc[0:5, :].reset_index()

# Flatten multi-index column headers from the aggregating functions above, combining the header levels with an underscore
gross_director_plot_data.columns = gross_director_plot_data.columns.map('_'.join)

# Plot the data and display!
gross_director_plot = (alt.Chart(gross_director_plot_data, width=500, height=300)
                       .mark_bar()
                       .encode(x=alt.X('director_:N', title="Director", sort="-y"),
                               y=alt.Y('Inflation_adjusted_gross_sum:Q', title="Inflation Adjusted Gross ($M USD)"))
                       .properties(title="Chart 2: Top 5 Disney Movie Directors Ranked by Sum of Inflation Adjusted Gross Revenue (1937-2016)"))
gross_director_plot

alt.Chart(...)

## Discussion

In this analysis, I looked at the Disney datasets to answer two main questions. One, how have the contributions of Disney's business segments changed over time, and is Disney Studios (one of the most recognizable elements of their company) their largest business? Two, who are the top 5 Disney directors ranked by total gross revenue, and in which time periods did they direct?

For the first question, I discovered that **Disney Studios** is only the third largest business segment since 1991, ranking behind the **Media Networks** and **Parks & Resorts** segments. In terms of growth, I confirmed my initial hypothesis that their digital focused business segments grew significantly more quickly than their more traditional business segments since 1997. Their **Interactive** (gaming) segment grew by almost 600%, and their **Media Networks** segment grew by approximately 250%. This compares very favourably to their **Total** overall revenue growth of approximately 130% since 1997. I would expect that Disney will continue to invest more money into these fast growing business segments to maximize revenue growth.

For the second question, I was surprised to find that the highest revenue directors were skewed towards Disney's older films. None of the top 5 directors started directing movies after 2000, and 4 of the 5 directors started directing before 1970. Note that "full credits" is a missing data point that refers to the movie Fantasia in 1940. Another interesting phenomenon is that **Wolfgang Reitherman** appears to have directed movies for 55 years, however that is due to re-releases of older movies. A potential future analysis could remove those re-releases from the total gross revenue and compare only the first release of each unique movie title to more fairly compare the directors. This analysis could potentially identify whether modernizing older films generates more revenue than creating new original films.

## References
* All the work in this project is original. Data sources are listed below.
### Resources Used
* [Disney Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * Dataset created by Kelly Garrett, hosted on data.world